In [2]:
import tensorflow as tf
import numpy as np

### 1.1 build model from block
- tf.keras.Model类是tf.keras模块里提供的一个模型构造类，我们可以继承它来定义我们想要的模型。
- 下面定义的MLP类重载了tf.keras.Model类的__init__函数和call函数。它们分别用于创建模型参数和定义前向计算。

In [5]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()#Flatten层将除第一维（batch_size）以外的维度展平
        self.dence1 = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)
        
    def call(self, inputs):
        x = self.flatten(inputs)
        x = self.dence1(x)
        output = self.dense2(x)
        return output

以上的MLP类中无须定义反向传播函数。系统将通过自动求梯度而自动生成反向传播所需的backward函数。

实例化MLP类得到模型变量net。

下面的代码初始化net并传入输入数据X做一次前向计算。

其中，net(X)将调用MLP类定义的call函数来完成前向计算

In [3]:
x = tf.random.uniform((2,20))

In [4]:
x

<tf.Tensor: shape=(2, 20), dtype=float32, numpy=
array([[0.12335026, 0.71435344, 0.6334133 , 0.40966487, 0.13752842,
        0.7016542 , 0.81597066, 0.61251533, 0.41511297, 0.2967198 ,
        0.8559363 , 0.90746164, 0.773767  , 0.9487393 , 0.05601525,
        0.04543793, 0.39873564, 0.23888302, 0.5986916 , 0.87822306],
       [0.12450159, 0.27776515, 0.103459  , 0.3578192 , 0.5973673 ,
        0.34271777, 0.40165794, 0.6890608 , 0.20921326, 0.1903454 ,
        0.356449  , 0.695385  , 0.9641104 , 0.8011559 , 0.5936271 ,
        0.9520819 , 0.773932  , 0.53147376, 0.01793122, 0.48219693]],
      dtype=float32)>

In [8]:
net = MLP()
net(x)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.43565702,  0.02873291,  0.0973721 , -0.38374364, -0.25873524,
        -0.3590687 ,  0.27803087,  0.37576792,  0.09905621,  0.08318231],
       [-0.05725191, -0.08459843, -0.10647976, -0.12196989, -0.21263993,
        -0.39308506,  0.13101135,  0.21821958,  0.04415824,  0.06083668]],
      dtype=float32)>

### 1.2 Sequential
- tf.keras.Model类是一个通用的部件。事实上，Sequential类继承自tf.keras.Model类。当模型的前向计算为简单串联各个层的计算时，可以通过更加简单的方式定义模型。这正是Sequential类的目的：它提供add函数来逐一添加串联的Block子类实例，而模型的前向计算就是将这些实例按添加的顺序逐一计算。

- 用Sequential类来实现前面描述的MLP类，并使用随机初始化的模型做一次前向计算。
- Sequential类可以使模型构造更加简单，且不需要定义call函数，但直接继承tf.keras.Model类可以极大地拓展模型构造的灵活性。

In [9]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
])
model(x)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.08006182, -0.00380369, -0.11005731,  0.13930875,  0.04617419,
         0.05314414,  0.30083013, -0.01032163, -0.10201715, -0.13135563],
       [ 0.18882011,  0.06640825,  0.08350617, -0.02319756,  0.01685662,
         0.00275064,  0.19363008,  0.09534745, -0.04183264,  0.06247529]],
      dtype=float32)>

### 1.3 build complex model
- 构造一个稍微复杂点的网络FancyMLP。在这个网络中，我们通过constant函数创建训练中不被迭代的参数，即常数参数。在前向计算中，除了使用创建的常数参数外，我们还使用tensor的函数和Python的控制流，并多次调用相同的层。

In [10]:
class FancyMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.rand_weight = tf.constant(
            tf.random.uniform((20,20)))
        self.dense = tf.keras.layers.Dense(units=20, activation=tf.nn.relu)

    def call(self, inputs):         
        x = self.flatten(inputs)   
        x = tf.nn.relu(tf.matmul(x, self.rand_weight) + 1)
        x = self.dense(x)    
        while tf.norm(x) > 1:
            x /= 2
        if tf.norm(x) < 0.8:
            x *= 10
        return tf.reduce_sum(x)

在这个FancyMLP模型中，我们使用了常数权重rand_weight（注意它不是模型参数）、做了矩阵乘法操作（tf.matmul）并重复使用了相同的Dense层。

In [12]:
net = FancyMLP()
net(x)

<tf.Tensor: shape=(), dtype=float32, numpy=3.1951232>

因为FancyMLP和Sequential类都是tf.keras.Model类的子类，所以我们可以嵌套调用它们。

In [14]:
class NestMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.net = tf.keras.Sequential()
        self.net.add(tf.keras.layers.Flatten())
        self.net.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
        self.net.add(tf.keras.layers.Dense(32, activation=tf.nn.relu))
        self.dense = tf.keras.layers.Dense(units=16, activation=tf.nn.relu)

    
    def call(self, inputs):         
        return self.dense(self.net(inputs))

net = tf.keras.Sequential()
net.add(NestMLP())
net.add(tf.keras.layers.Dense(20))
net.add(FancyMLP())

net(x)

<tf.Tensor: shape=(), dtype=float32, numpy=25.283884>

### 2.模型参数的访问、初始化和共享
- 本节将深入讲解如何访问和初始化模型参数，以及如何在多个层之间共享同一份模型参数。

In [15]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
])
model(x)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.28309575,  0.33035207,  0.00477698,  0.38263905, -0.06752132,
         0.18543263, -0.00471738,  0.14215837,  0.14404565,  0.14132789],
       [-0.20838147,  0.26619714, -0.1374893 ,  0.35440785,  0.06613217,
         0.07102975,  0.3096976 , -0.03204616,  0.08957964,  0.2130332 ]],
      dtype=float32)>

#### 2.1 access model parameters
- 对于使用Sequential类构造的神经网络，我们可以通过weights属性来访问网络任一层的权重。

In [17]:
model.weights[0]

<tf.Variable 'dense_16/kernel:0' shape=(20, 256) dtype=float32, numpy=
array([[-0.01903628, -0.14045799,  0.06267899, ..., -0.03502336,
         0.14657515, -0.13448143],
       [ 0.13456693, -0.02639709,  0.01701823, ..., -0.04758588,
         0.13559869, -0.01687135],
       [-0.12671478, -0.0481895 ,  0.08373944, ..., -0.10435146,
        -0.09566057,  0.14666396],
       ...,
       [ 0.12309813, -0.04828718, -0.04760803, ..., -0.14705625,
         0.03321984,  0.04508564],
       [ 0.04203133, -0.02572612,  0.10581619, ...,  0.05057153,
        -0.05203209, -0.05922529],
       [-0.10549372,  0.11880544, -0.1323233 , ...,  0.1088658 ,
        -0.1132108 , -0.08223248]], dtype=float32)>

#### 2.2initialize params
- 在下面的例子中，我们将权重参数初始化成均值为0、标准差为0.01的正态分布随机数，并依然将偏差参数清零。

In [ ]:
class Linear(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.d1 = tf.keras.layers.Dense(
            units=10,
            activation=None,
            kernel_initializer=tf.random_normal_initializer(mean=0,stddev=0.01),
            bias_initializer=tf.zeros_initializer()
        )
        self.d2 = tf.keras.layers.Dense(
            units=1,
            activation=None,
            kernel_initializer=tf.ones_initializer(),
            bias_initializer=tf.ones_initializer()
        )

    def call(self, input):
        output = self.d1(input)
        output = self.d2(output)
        return output

net = Linear()
net(x)
net.get_weights()

#### 2.3 define initializer
- 可以使用tf.keras.initializers类中的方法实现自定义初始化。

In [18]:
def my_init():
    return tf.keras.initializers.Ones()

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(64, kernel_initializer=my_init()))

Y = model(x)
model.weights[0]

<tf.Variable 'dense_18/kernel:0' shape=(20, 64) dtype=float32, numpy=
array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)>

### 3.自定义层

深度学习的一个魅力在于神经网络中各式各样的层，例如全连接层和后面章节中将要介绍的卷积层、池化层与循环层。虽然tf.keras提供了大量常用的层，但有时候我们依然希望自定义层。本节将介绍如何自定义一个层，从而可以被重复调用。

#### 3.1 custom layer without parameters
CenteredLayer类通过继承tf.keras.layers.Layer类自定义了一个将输入减掉均值后输出的层，并将层的计算定义在了call函数里。这个层里不含模型参数。

In [19]:
class CenteredLayer(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()

    def call(self, inputs):
        return inputs - tf.reduce_mean(inputs)

我们可以实例化这个层，然后做前向计算。

In [20]:
layer = CenteredLayer()
layer(np.array([1,2,3,4,5]))

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([-2, -1,  0,  1,  2])>

我们也可以用它来构造更复杂的模型。

In [21]:
net = tf.keras.models.Sequential()
net.add(tf.keras.layers.Flatten())
net.add(tf.keras.layers.Dense(20))
net.add(CenteredLayer())

Y = net(x)
Y

<tf.Tensor: shape=(2, 20), dtype=float32, numpy=
array([[-0.18561336,  0.7980589 ,  0.5337747 , -0.86443174,  0.13248481,
         0.32119542, -0.5011614 ,  0.5986253 ,  0.24122542, -0.22686513,
         0.38301235,  0.11574471,  0.7210884 ,  0.28445512, -0.2951304 ,
        -0.96856207,  0.8118604 , -0.42314735, -0.11699405,  0.2758397 ],
       [-0.21926036, -0.08459637,  0.3569438 , -0.5331099 , -0.34114537,
         0.4048779 , -0.92176265,  0.07205981, -0.42045605, -0.32289994,
         0.8264247 ,  0.1391777 ,  0.6410753 , -0.5132173 , -0.10635653,
        -0.37071216,  0.6091706 , -0.41690716, -0.38988742, -0.04487823]],
      dtype=float32)>

#### 3.2 custom layer with parameters
- 还可以自定义含模型参数的自定义层。其中的模型参数可以通过训练学出。

In [22]:
class myDense(tf.keras.layers.Layer):
    def __init__(self, units):
        super().__init__()
        self.units = units

    def build(self, input_shape):     # 这里 input_shape 是第一次运行call()时参数inputs的形状
        self.w = self.add_weight(name='w',
            shape=[input_shape[-1], self.units], initializer=tf.random_normal_initializer())
        self.b = self.add_weight(name='b',
            shape=[self.units], initializer=tf.zeros_initializer())

    def call(self, inputs):
        y_pred = tf.matmul(inputs, self.w) + self.b
        return y_pred

实例化MyDense类并访问它的模型参数

In [23]:
dense = myDense(3)
dense(x)
dense.get_weights()

[array([[-0.0662183 , -0.07223793, -0.02863855],
        [ 0.05602537, -0.01299378,  0.02967717],
        [ 0.07740184,  0.00111303, -0.02963178],
        [ 0.00070688,  0.00184615,  0.00323411],
        [-0.04707849, -0.0902397 , -0.04093401],
        [-0.05739418,  0.06970806, -0.04940173],
        [ 0.01465408,  0.09121215,  0.04880067],
        [-0.00669081, -0.00186088, -0.10201043],
        [ 0.03363297, -0.03407352,  0.0095716 ],
        [-0.04044167, -0.09362209, -0.09052042],
        [-0.00043173,  0.04064349,  0.04312221],
        [ 0.00928534,  0.04488818, -0.1534695 ],
        [ 0.02520572, -0.00927218,  0.06498551],
        [ 0.00830077, -0.02744145,  0.05858869],
        [ 0.01789406, -0.04866096, -0.00970302],
        [ 0.00092342, -0.01169716, -0.04284624],
        [-0.03877211,  0.01373667,  0.08019301],
        [ 0.01347374, -0.03372454,  0.03266772],
        [ 0.09045189,  0.06500303,  0.0539767 ],
        [ 0.03045775, -0.06434947, -0.04761484]], dtype=float32),
 ar

我们也可以使用自定义层构造模型。

In [25]:
net = tf.keras.models.Sequential()
net.add(myDense(8))
net.add(myDense(1))

net(x)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.00360968],
       [0.0185812 ]], dtype=float32)>